In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.express as px
# import plotly.io as pio
import re
# import rosbag_pandas
import math
import yaml
import pickle
import PyQt5
# from rosbag.bag import Bag

In [3]:
def failureProbability_linAcc(acc_z):
    sigma1 = 0.4
    sigma2 = -0.9
    mu = 0
    output = 1 - (1/(math.sqrt(2*math.pi)*sigma1))*(math.exp( - math.pow(acc_z - mu,2)/(2*math.pow(sigma2,2))))
    if(output>1):
        return 1
    elif(output<0):
        return 0
    else:
        return output

def magROC_cdf(x):
    a = -6
    b = -0.15
    try:
        p_f = 1/(1 + math.exp(-a*x + a*b))
    except OverflowError:
        p_f = float(0)
#         print('the values that caused this problem is:',x)
    return p_f


def odomError_cdf(timeSteps):
    k = 0.2
    p_f = k*timeSteps
    #This is a threshold to make the graphs clearer
    if(p_f>1):
        return 1
    elif(p_f<0):
        return 0
    else:
        return p_f

def vel_cdf(x):
    a = 1.5
    b = 2.5
    try:
        p_f = 1/(1 + math.exp(-a*x + a*b))
    except OverflowError:
        p_f = float(0)
    return p_f


def psnrError_cdf(x):
    a = 5
    b = 1
    try:
        p_f = 1/(1 + math.exp(-a*x + a*b))
    except OverflowError:
        p_f = float(0)
#         print('the values that caused this problem is:',x)
    return p_f

def probabilityOfFailure(arrayOfProbabilities):
    normalisationConstant = 1.0/(len(arrayOfProbabilities))
    p_f_total = 0.0
    for p_f in arrayOfProbabilities:
        p_f_total = p_f_total + p_f*normalisationConstant
    return p_f_total

In [4]:
def dataframe_parser(path_to_file, path_to_infoDict, arena, difficulty,trial):
#     print('Parsing Rosbag ',path_to_file)
#     df = rosbag_pandas.bag_to_dataframe(path_to_file)
#     info_dict = yaml.load(Bag(path_to_file, 'r')._get_yaml_info())
    # df.head()
    print('Parsing pickle file', path_to_file)
    df = pd.read_pickle(path_to_file)
    info_dict = pickle.load(open(path_to_infoDict, "rb"))
    timeToComplete = info_dict['duration']
    
    rv_columns = ['move_base_current_goal__pose_position_x',
                  'move_base_current_goal__pose_position_y',
                  'move_base_current_goal__pose_position_z',
                  'husky_velocity_controller_odom__pose_pose_position_x',
                  'husky_velocity_controller_odom__pose_pose_position_y',
                  'husky_velocity_controller_odom__pose_pose_position_z',
                  'odometry_filtered__pose_pose_position_x',
                  'odometry_filtered__pose_pose_position_y',
                  'odometry_filtered__pose_pose_position_z',
                  'imu_data__linear_acceleration_z',
                  'psnr_laserScan__data']

    df = df.fillna(method = 'ffill')
    df = df.fillna(0)
    df.describe()
    df['arena'] = arena
    df['difficulty'] = difficulty
    df['trial'] = trial

    df['move_base_current_goal__pose_position_x'] = 16.3375
    df['move_base_current_goal__pose_position_y']= 0.5000
    df['move_base_current_goal__pose_position_z']= 0.0000
    
    df['move_base_current_goal__pose_position_x'] = df['move_base_current_goal__pose_position_x'].apply(lambda x: np.float32(16.3375) if(x == 0) else None)
        
    df['odom_diff_x'] = df['husky_velocity_controller_odom__pose_pose_position_x'] - df['odometry_filtered__pose_pose_position_x']
    df['odom_diff_y'] = df['husky_velocity_controller_odom__pose_pose_position_y'] - df['odometry_filtered__pose_pose_position_y']
    df['odom_diff_z'] = df['husky_velocity_controller_odom__pose_pose_position_z'] - df['odometry_filtered__pose_pose_position_z']
    df['odom_posErr'] = np.linalg.norm(df[['odom_diff_x','odom_diff_y','odom_diff_z']].values, axis=1)

    df['magDist_diff_x'] = df['move_base_current_goal__pose_position_x'] - df['odometry_filtered__pose_pose_position_x']
    df['magDist_diff_y'] = df['move_base_current_goal__pose_position_y'] - df['odometry_filtered__pose_pose_position_y']
    df['magDist_diff_z'] = df['move_base_current_goal__pose_position_z'] - df['odometry_filtered__pose_pose_position_z']
    df['magDist_diff_x'] = df['magDist_diff_x'].astype('float')
    df['magDist_diff_y'] = df['magDist_diff_y'].astype('float')
    df['magDist_diff_z'] = df['magDist_diff_z'].astype('float')
    df['mag_distFromGoal'] = np.linalg.norm(df[['magDist_diff_x','magDist_diff_y','magDist_diff_z']].values, axis=1)
    
    df = df.reset_index()
    df = df.drop(columns = ['index'])
    samplingRate = 240
#     df.describe()
    df_sampled = pd.DataFrame(columns = ['imu_data__linear_acceleration_z','mag_distFromGoal','odom_posErr','psnr_laserScan__data','odometry_filtered__pose_pose_position_x','odometry_filtered__pose_pose_position_y','odometry_filtered__pose_pose_position_z','arena','difficulty','trial'])
    for i in range(0,len(df),samplingRate):
        df_sampled = df_sampled.append(df[['imu_data__linear_acceleration_z','mag_distFromGoal','odom_posErr','psnr_laserScan__data','odometry_filtered__pose_pose_position_x','odometry_filtered__pose_pose_position_y','odometry_filtered__pose_pose_position_z','arena','difficulty','trial']].iloc[i,:],ignore_index = True)
#     df_sampled = df_sampled.drop(index=df_sampled.index[:5],axis=0)
    
#     df_sampled.describe()

    df_sampled['mag_distFromGoal_roc'] = df_sampled['mag_distFromGoal'].diff(periods = 3).fillna(0)
    df_sampled['imu_data__linear_acceleration_z'].iloc[0] = df_sampled['imu_data__linear_acceleration_z'].iloc[1]
    # df_sampled['mag_distFromGoal_roc'] = df_sampled['mag_distFromGoal_roc'].fillna(0)
    df_sampled['imu_linAcc_z_roc'] = df_sampled['imu_data__linear_acceleration_z'].diff(2).fillna(method='bfill')
    df_sampled['odom_posErr_roc'] = df_sampled['odom_posErr'].diff().fillna(0)
#     df_sampled['psnr_laserScan__roc'] = df_sampled['psnr_laserScan__data'].diff(3).fillna(0)
#     df_sampled['psnr_laserScan__roc'].iloc[0] = df_sampled['psnr_laserScan__roc'].iloc[1]
    # df_sampled.head()
#     print('Came here')
    df_sampled['magROC_event'] = 500
    
    corr_vector = np.concatenate((np.linspace(0,-2.5,5), np.linspace(-2.5,0,6)[1:]), axis=None)
    for i in range(0,len(df_sampled['mag_distFromGoal_roc']) - len(corr_vector)):
        sum = 0
        for j in range(0,len(corr_vector)):
            sum = sum + (df_sampled['mag_distFromGoal_roc'].iloc[i+j] * corr_vector[len(corr_vector)-1 -j])
#         print('Im here bro')
        df_sampled['magROC_event'].iloc[i] = sum

#     corr_vector = np.linspace(-2.5,2.5,5)
#     for i in range(5,len(df_sampled['psnr_laserScan__roc']) - len(corr_vector)):
#         sum = 0
#         for j in range(0,len(corr_vector)):
#             sum = sum + (df_sampled['psnr_laserScan__roc'].iloc[i+j] * corr_vector[len(corr_vector)-1 -j])
# #         print('Im here bro')
#         df_sampled['psnr_event'].iloc[i] = sum    
   
    df_sampled['velocity_event'] = 0
    df_sampled['velocity_event_count'] = 0
    df_sampled['odom_mag'] = np.linalg.norm(df_sampled[['odometry_filtered__pose_pose_position_x','odometry_filtered__pose_pose_position_y','odometry_filtered__pose_pose_position_z']].values, axis=1)
    df_sampled['velocity_fromOdom'] = df_sampled['odom_mag'].diff(periods = 3)  
    df_sampled['velocity_event'] = df_sampled['velocity_fromOdom'].apply(lambda x: 1 if (x<0.01) else 0)
    a = df_sampled['velocity_event']!= 0
    df_sampled['velocity_event_count'] = a.cumsum()-a.cumsum().where(~a).ffill().fillna(0).astype(int)
    

#     i = 5
#     df_sampled['psnr_event'] = 0
#     while(i in range (5,len(df_sampled['psnr_laserScan__roc']) - 1)):
#         if(df_sampled['psnr_laserScan__roc'].iloc[i]<0.1):
#             count = 1
#             while(abs(df_sampled['psnr_laserScan__roc'].iloc[i+1] - df_sampled['psnr_laserScan__roc'].iloc[i])<0.1):
#                 df_sampled['psnr_event'].iloc[i] = count
#                 count = count + 1
#                 i = i+1
#         i = i+1
 
    df_sampled['odom_event'] = 0

    i = 1
    while(i in range(5,len(df_sampled['odom_posErr_roc']) - 1)):
    #     print('loop entered')
        if(((i+1)< len(df_sampled['odom_posErr_roc'])) and 
             (abs(df_sampled['odom_posErr_roc'].iloc[i]) < 0.001 and abs(df_sampled['odom_posErr_roc'].iloc[i+1]) < 0.001)
            ):
            count = 1
    #         print(i)
            while(((i+1)< len(df_sampled['odom_posErr_roc'])) and (abs(df_sampled['odom_posErr_roc'].iloc[i+1]) < 0.001)):
                df_sampled['odom_event'].iloc[i] = count
                count = count+1
                i = i+1
        i = i+1
    df_sampled['rv_magROC_event'] = df_sampled['magROC_event'].apply(magROC_cdf)
    df_sampled['rv_vel_event'] = df_sampled['velocity_event_count'].apply(vel_cdf)
    df_sampled['rv_odom_event'] = df_sampled['odom_event'].apply(odomError_cdf)
    df_sampled['rv_linAcc_Z_event'] = df_sampled['imu_linAcc_z_roc'].apply(failureProbability_linAcc)
    df_sampled['rv_snr_event'] = df_sampled['psnr_laserScan__data'].apply(psnrError_cdf)
#     df_sampled['p_f_total'] = df_sampled.apply(lambda x: probabilityOfFailure(np.array([x['rv_magROC_event'],x['rv_linAcc_Z_event'],x['rv_snr_event'],x['rv_vel_event']])),axis = 1)
    df_sampled['p_f_total'] = df_sampled.apply(lambda x: probabilityOfFailure(np.array([x['rv_magROC_event'],x['rv_odom_event'],x['rv_linAcc_Z_event'],x['rv_snr_event'],x['rv_vel_event']])),axis = 1)
    df_sampled['robot_health'] = (df_sampled['p_f_total']*np.log10(df_sampled['p_f_total'])).rolling(10).sum()
    df_sampled = df_sampled.reset_index()
    mean_health = df_sampled['robot_health'].mean()
    return [mean_health,timeToComplete,arena,difficulty,trial]

In [49]:
def dataframe_parser_healthTrend(path_to_file, path_to_infoDict, arena, difficulty,trial):
#     print('Parsing Rosbag ',path_to_file)
#     df = rosbag_pandas.bag_to_dataframe(path_to_file)
#     info_dict = yaml.load(Bag(path_to_file, 'r')._get_yaml_info())
    # df.head()
    print('Parsing pickle file', path_to_file)
    df = pd.read_pickle(path_to_file)
    info_dict = pickle.load(open(path_to_infoDict, "rb"))
    timeToComplete = info_dict['duration']
    
    rv_columns = ['move_base_current_goal__pose_position_x',
                  'move_base_current_goal__pose_position_y',
                  'move_base_current_goal__pose_position_z',
                  'husky_velocity_controller_odom__pose_pose_position_x',
                  'husky_velocity_controller_odom__pose_pose_position_y',
                  'husky_velocity_controller_odom__pose_pose_position_z',
                  'odometry_filtered__pose_pose_position_x',
                  'odometry_filtered__pose_pose_position_y',
                  'odometry_filtered__pose_pose_position_z',
                  'imu_data__linear_acceleration_z',
                  'psnr_laserScan__data']

    df = df.fillna(method = 'ffill')
    df = df.fillna(0)
    df.describe()
    df['arena'] = arena
    df['difficulty'] = difficulty
    df['trial'] = trial

    df['move_base_current_goal__pose_position_x'] = 16.3375
    df['move_base_current_goal__pose_position_y']= 0.5000
    df['move_base_current_goal__pose_position_z']= 0.0000
    
    df['move_base_current_goal__pose_position_x'] = df['move_base_current_goal__pose_position_x'].apply(lambda x: np.float32(16.3375) if(x == 0) else None)
        
    df['odom_diff_x'] = df['husky_velocity_controller_odom__pose_pose_position_x'] - df['odometry_filtered__pose_pose_position_x']
    df['odom_diff_y'] = df['husky_velocity_controller_odom__pose_pose_position_y'] - df['odometry_filtered__pose_pose_position_y']
    df['odom_diff_z'] = df['husky_velocity_controller_odom__pose_pose_position_z'] - df['odometry_filtered__pose_pose_position_z']
    df['odom_posErr'] = np.linalg.norm(df[['odom_diff_x','odom_diff_y','odom_diff_z']].values, axis=1)

    df['magDist_diff_x'] = df['move_base_current_goal__pose_position_x'] - df['odometry_filtered__pose_pose_position_x']
    df['magDist_diff_y'] = df['move_base_current_goal__pose_position_y'] - df['odometry_filtered__pose_pose_position_y']
    df['magDist_diff_z'] = df['move_base_current_goal__pose_position_z'] - df['odometry_filtered__pose_pose_position_z']
    df['magDist_diff_x'] = df['magDist_diff_x'].astype('float')
    df['magDist_diff_y'] = df['magDist_diff_y'].astype('float')
    df['magDist_diff_z'] = df['magDist_diff_z'].astype('float')
    df['mag_distFromGoal'] = np.linalg.norm(df[['magDist_diff_x','magDist_diff_y','magDist_diff_z']].values, axis=1)
    
    df = df.reset_index()
    df = df.drop(columns = ['index'])
#     samplingRate = 240
    samplingRate = int(len(df.index)/timeToComplete)
#     df.describe()
    df_sampled = pd.DataFrame(columns = ['imu_data__linear_acceleration_z','mag_distFromGoal','odom_posErr','psnr_laserScan__data','odometry_filtered__pose_pose_position_x','odometry_filtered__pose_pose_position_y','odometry_filtered__pose_pose_position_z','arena','difficulty','trial'])
    for i in range(0,len(df),samplingRate):
        df_sampled = df_sampled.append(df[['imu_data__linear_acceleration_z','mag_distFromGoal','odom_posErr','psnr_laserScan__data','odometry_filtered__pose_pose_position_x','odometry_filtered__pose_pose_position_y','odometry_filtered__pose_pose_position_z','arena','difficulty','trial']].iloc[i,:],ignore_index = True)
#     df_sampled = df_sampled.drop(index=df_sampled.index[:5],axis=0)
    
#     df_sampled.describe()

    df_sampled['mag_distFromGoal_roc'] = df_sampled['mag_distFromGoal'].diff(periods = 3).fillna(0)
    df_sampled['imu_data__linear_acceleration_z'].iloc[0] = df_sampled['imu_data__linear_acceleration_z'].iloc[1]
    # df_sampled['mag_distFromGoal_roc'] = df_sampled['mag_distFromGoal_roc'].fillna(0)
    df_sampled['imu_linAcc_z_roc'] = df_sampled['imu_data__linear_acceleration_z'].diff(2).fillna(method='bfill')
    df_sampled['odom_posErr_roc'] = df_sampled['odom_posErr'].diff().fillna(0)
#     df_sampled['psnr_laserScan__roc'] = df_sampled['psnr_laserScan__data'].diff(3).fillna(0)
#     df_sampled['psnr_laserScan__roc'].iloc[0] = df_sampled['psnr_laserScan__roc'].iloc[1]
    # df_sampled.head()
#     print('Came here')
    df_sampled['magROC_event'] = 500
    
    corr_vector = np.concatenate((np.linspace(0,-2.5,5), np.linspace(-2.5,0,6)[1:]), axis=None)
    for i in range(0,len(df_sampled['mag_distFromGoal_roc']) - len(corr_vector)):
        sum = 0
        for j in range(0,len(corr_vector)):
            sum = sum + (df_sampled['mag_distFromGoal_roc'].iloc[i+j] * corr_vector[len(corr_vector)-1 -j])
#         print('Im here bro')
        df_sampled['magROC_event'].iloc[i] = sum

#     corr_vector = np.linspace(-2.5,2.5,5)
#     for i in range(5,len(df_sampled['psnr_laserScan__roc']) - len(corr_vector)):
#         sum = 0
#         for j in range(0,len(corr_vector)):
#             sum = sum + (df_sampled['psnr_laserScan__roc'].iloc[i+j] * corr_vector[len(corr_vector)-1 -j])
# #         print('Im here bro')
#         df_sampled['psnr_event'].iloc[i] = sum    
   
    df_sampled['velocity_event'] = 0
    df_sampled['velocity_event_count'] = 0
    df_sampled['odom_mag'] = np.linalg.norm(df_sampled[['odometry_filtered__pose_pose_position_x','odometry_filtered__pose_pose_position_y','odometry_filtered__pose_pose_position_z']].values, axis=1)
    df_sampled['velocity_fromOdom'] = df_sampled['odom_mag'].diff(periods = 3)  
    df_sampled['velocity_event'] = df_sampled['velocity_fromOdom'].apply(lambda x: 1 if (x<0.01) else 0)
    a = df_sampled['velocity_event']!= 0
    df_sampled['velocity_event_count'] = a.cumsum()-a.cumsum().where(~a).ffill().fillna(0).astype(int)
    

#     i = 5
#     df_sampled['psnr_event'] = 0
#     while(i in range (5,len(df_sampled['psnr_laserScan__roc']) - 1)):
#         if(df_sampled['psnr_laserScan__roc'].iloc[i]<0.1):
#             count = 1
#             while(abs(df_sampled['psnr_laserScan__roc'].iloc[i+1] - df_sampled['psnr_laserScan__roc'].iloc[i])<0.1):
#                 df_sampled['psnr_event'].iloc[i] = count
#                 count = count + 1
#                 i = i+1
#         i = i+1
 
    df_sampled['odom_event'] = 0

    i = 1
    while(i in range(5,len(df_sampled['odom_posErr_roc']) - 1)):
    #     print('loop entered')
        if(((i+1)< len(df_sampled['odom_posErr_roc'])) and 
             (abs(df_sampled['odom_posErr_roc'].iloc[i]) < 0.001 and abs(df_sampled['odom_posErr_roc'].iloc[i+1]) < 0.001)
            ):
            count = 1
    #         print(i)
            while(((i+1)< len(df_sampled['odom_posErr_roc'])) and (abs(df_sampled['odom_posErr_roc'].iloc[i+1]) < 0.001)):
                df_sampled['odom_event'].iloc[i] = count
                count = count+1
                i = i+1
        i = i+1
    df_sampled['rv_magROC_event'] = df_sampled['magROC_event'].apply(magROC_cdf)
    df_sampled['rv_vel_event'] = df_sampled['velocity_event_count'].apply(vel_cdf)
    df_sampled['rv_odom_event'] = df_sampled['odom_event'].apply(odomError_cdf)
    df_sampled['rv_linAcc_Z_event'] = df_sampled['imu_linAcc_z_roc'].apply(failureProbability_linAcc)
    df_sampled['rv_snr_event'] = df_sampled['psnr_laserScan__data'].apply(psnrError_cdf)
#     df_sampled['p_f_total'] = df_sampled.apply(lambda x: probabilityOfFailure(np.array([x['rv_magROC_event'],x['rv_linAcc_Z_event'],x['rv_snr_event'],x['rv_vel_event']])),axis = 1)
    df_sampled['p_f_total'] = df_sampled.apply(lambda x: probabilityOfFailure(np.array([x['rv_magROC_event'],x['rv_odom_event'],x['rv_linAcc_Z_event'],x['rv_snr_event'],x['rv_vel_event']])),axis = 1)
    df_sampled['robot_health'] = -(df_sampled['p_f_total']*np.log10(df_sampled['p_f_total'])).rolling(10).sum()
    df_sampled = df_sampled.reset_index()
    mean_health = df_sampled['robot_health'].mean()
    return [mean_health,timeToComplete,arena,difficulty,trial,df_sampled['robot_health'].values]

In [50]:
import os
directory = r'/Volumes/Aniketh/rosbags/NovemberTrial_1'
# n = len(os.listdir(directory))
matrix = []
# count = 0
for filename in os.listdir(directory):
    if filename.endswith('bag'):
        df_filename = filename[:-3]+'pickle'
        info_dict_filename = filename[:-4]+'__info_dict.pickle'
        myList = dataframe_parser(os.path.join(directory, df_filename),os.path.join(directory, info_dict_filename),filename.split('_')[0],filename.split('_')[1],filename.split('_')[2][:-4])
        matrix.append(myList)
#         matrix[count,1] = myArray[1]
#         count = count + 1

df = pd.DataFrame(matrix,columns = ['average_robotHealth','timeTaken','Arena','Difficulty','Trial'])


Parsing pickle file /Volumes/Aniketh/rosbags/NovemberTrial_1/obstacles_0_1.pickle


/Users/anikethramesh/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Parsing pickle file /Volumes/Aniketh/rosbags/NovemberTrial_1/obstacles_0_2.pickle


/Users/anikethramesh/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Parsing pickle file /Volumes/Aniketh/rosbags/NovemberTrial_1/obstacles_0_3.pickle


...

/Users/anikethramesh/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Parsing pickle file /Volumes/Aniketh/rosbags/NovemberTrial_1/obstaclesRoughTerrainLaserNoise_hard_4.pickle


/Users/anikethramesh/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Parsing pickle file /Volumes/Aniketh/rosbags/NovemberTrial_1/obstaclesRoughTerrainLaserNoise_hard_5.pickle


/Users/anikethramesh/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [33]:
df['Arena'] = df['Arena'].replace('obstacles','Uneven Terrain')
df['Arena'] = df['Arena'].replace('snr','Laser Noise')
df['Arena'] = df['Arena'].replace('obstaclesRoughTerrainLaserNoise','Noise and Uneven Terrain')


In [34]:
df

,average_robotHealth,timeTaken,Arena,Difficulty,Trial
0,-0.728020,42.635945,Uneven Terrain,0,1
1,-0.691093,40.567000,Uneven Terrain,0,2
2,-0.685066,39.631885,Uneven Terrain,0,3
3,-0.692744,39.702954,Uneven Terrain,0,4
4,-0.686723,39.507461,Uneven Terrain,0,5
5,-1.050454,64.546000,Uneven Terrain,10,1
6,-1.269821,108.354245,Uneven Terrain,10,2
7,-1.229904,131.301577,Uneven Terrain,10,3
8,-1.169083,109.853087,Uneven Terrain,10,4
9,-1.258128,184.391000,Uneven Terrain,10,5


In [41]:
df.loc[(df.Arena == 'Uneven Terrain') & (df.Difficulty == '0'), 'Level'] = 'Baseline'
df.loc[(df.Arena == 'Laser Noise') & (df.Difficulty == '0'), 'Level'] = 'Baseline'

df.loc[(df.Arena == 'Uneven Terrain') & (df.Difficulty == '10'), 'Level'] = 'Level 1'
df.loc[(df.Arena == 'Uneven Terrain') & (df.Difficulty == '20'), 'Level'] = 'Level 2'
df.loc[(df.Arena == 'Uneven Terrain') & (df.Difficulty == '40'), 'Level'] = 'Level 3'

df.loc[(df.Arena == 'Laser Noise') & (df.Difficulty == '04'), 'Level'] = 'Level 1'
df.loc[(df.Arena == 'Laser Noise') & (df.Difficulty == '05'), 'Level'] = 'Level 2'
df.loc[(df.Arena == 'Laser Noise') & (df.Difficulty == '06'), 'Level'] = 'Level 3'

df.loc[(df.Arena == 'Noise and Uneven Terrain') & (df.Difficulty == 'easy')  , 'Level'] = 'Level 1'
df.loc[(df.Arena == 'Noise and Uneven Terrain') & (df.Difficulty == 'medium'), 'Level'] = 'Level 2'
df.loc[(df.Arena == 'Noise and Uneven Terrain') & (df.Difficulty == 'hard'  ), 'Level'] = 'Level 3'

AttributeError: 'DataFrame' object has no attribute 'Arena'

In [42]:
df_levels = df.loc[df['Level'] != '']
df_levels

,average_robotHealth,timeTaken,Condition,Difficulty,Trial,Level
0,-0.728020,42.635945,Uneven Terrain,0,1,Baseline
1,-0.691093,40.567000,Uneven Terrain,0,2,Baseline
2,-0.685066,39.631885,Uneven Terrain,0,3,Baseline
3,-0.692744,39.702954,Uneven Terrain,0,4,Baseline
4,-0.686723,39.507461,Uneven Terrain,0,5,Baseline
5,-1.050454,64.546000,Uneven Terrain,10,1,Level 1
6,-1.269821,108.354245,Uneven Terrain,10,2,Level 1
7,-1.229904,131.301577,Uneven Terrain,10,3,Level 1
8,-1.169083,109.853087,Uneven Terrain,10,4,Level 1
9,-1.258128,184.391000,Uneven Terrain,10,5,Level 1


In [43]:
df = df.rename(columns={"Arena":"Condition"})
df_levels = df_levels.rename(columns={"Arena":"Condition"})

In [44]:
%matplotlib qt
ax =sns.scatterplot(data=df, x="timeTaken", y="average_robotHealth",hue='Condition',palette='deep')
ax.set_title('Correlation Plot for Exp I. Spearman\'s Rho Value = -0.96 ')
ax.set(ylabel = 'Average Robot Health over run time', xlabel = 'Time taken to complete the task')
# ax.legend(title='Condition',loc='upper right')

[Text(0, 0.5, 'Average Robot Health over run time'),
 Text(0.5, 0, 'Time taken to complete the task')]

In [45]:
corr_matrix = df[['timeTaken','average_robotHealth']].corr( method = 'spearman')

In [46]:
corr_matrix

,timeTaken,average_robotHealth
timeTaken,1.000000,-0.960761
average_robotHealth,-0.960761,1.000000


In [15]:
df['average_robotHealth'].values

array([-0.72802021, -0.69109307, -0.68506575, -0.69274364, -0.68672265,
       -1.05045442, -1.26982094, -1.22990448, -1.16908334, -1.25812757,
       -1.15859781, -1.17841809, -1.29442736, -0.81765331, -1.34863442,
       -1.25346235, -1.27324943, -1.0706015 , -1.50147025, -1.34183472,
       -0.72802021, -0.69109307, -0.68506575, -0.69274364, -0.68672265,
       -1.00040587, -1.16470415, -1.01084484, -0.97489243, -1.11858785,
       -1.21380156, -0.97639439, -0.98442257, -1.16791784, -1.09448883,
       -0.98280512, -1.04294752, -1.21798997, -1.07139737, -1.19490758,
       -1.15221374, -1.19402137, -1.14298898, -1.07318816, -1.10307555,
       -1.26723866, -1.16957269, -1.03644149, -1.27909548, -1.45315617,
       -1.20343342, -1.44055624, -1.12207597, -1.17254134, -1.15225589,
       -1.2612777 , -1.44267392, -1.44433746, -1.39334634, -1.39467172])

In [16]:
!python --version

Python 3.6.9 :: Anaconda, Inc.


In [47]:
%matplotlib qt
ax = sns.boxplot(x="Level", y="timeTaken",hue="Condition", data=df_levels)
ax.set_title('Time taken vs performance degradation in Exp I')
ax.set(xlabel = 'Levels of Performance degradation', ylabel = 'Time taken to complete the task')
# ax.legend(title='Condition',loc='upper left')

[Text(0.5, 0, 'Levels of Performance degradation'),
 Text(0, 0.5, 'Time taken to complete the task')]

In [54]:
%matplotlib qt
ax = sns.boxplot(x="Level", y="average_robotHealth",hue="Condition", data=df_levels)
ax.set_title('Run Time Average Robot Health vs performance degradation in Exp I')
ax.set(xlabel = 'Levels of Performance degradation', ylabel = 'Average Robot Health over run time')
# ax.legend(title='Condition',loc='upper left')

[Text(0.5, 0, 'Levels of Performance degradation'),
 Text(0, 0.5, 'Average Robot Health over run time')]

In [63]:
%matplotlib qt

fig, axs = plt.subplots(1,2)
sns.boxplot(x="Level", y="timeTaken",hue="Condition", data=df_levels, ax = axs[0])
axs[0].set_title('Time taken vs performance degradation in Exp I', fontsize='large')
axs[0].set_xlabel('Levels of Performance degradation',fontsize='large')
axs[0].set_ylabel('Time taken to complete the task (in seconds)',fontsize='large')
axs[0].legend(fontsize=14)

sns.boxplot(x="Level", y="average_robotHealth",hue="Condition", data=df_levels, ax = axs[1])
axs[1].set_title('Run Time Average Robot Health vs performance degradation in Exp I',fontsize='large')
axs[1].set_xlabel ('Levels of Performance degradation',fontsize='large')
axs[1].set_ylabel ('Average Robot Health over run time',fontsize='large')
axs[1].legend(fontsize=14)

In [63]:
df_levels.to_csv('sim_df.csv')

In [70]:
g = sns.catplot(x="sex", y="total_bill",hue="smoker", col="time",data=tips, kind="violin", split=True, height=4, aspect=.7);

In [83]:
%matplotlib qt
sns.pointplot(x="Level", y="timeTaken",hue="Condition", data=df_levels)

<AxesSubplot:xlabel='Level', ylabel='timeTaken'>